## 1- Installing prerequisites and cloning the project

In [ ]:
!pip install -q lmdb fire gdown
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

## 2- Downloading pretrained weights and dataset

In [ ]:
!gdown 1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM  #  فایل zip train
!gdown 1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I  #  فایل zip validation

# باز کردن zip ها
!unzip -q /content/plate_img-train.zip -d /content/Dataset/
!unzip -q /content/plate_img-validation.zip -d /content/Dataset/


## 3- Preparing GT (Ground Truth) files for training

In [ ]:
import os
import xml.etree.ElementTree as ET

train_dir = '/content/Dataset/train'
output_file = '/content/deep-text-recognition-benchmark/data/gt_train.txt'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
lines = []

for filename in os.listdir(train_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(train_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join('/content/deep-text-recognition-benchmark/data/train', image_name)

        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            xmin = int(obj.find('bndbox').find('xmin').text)
            chars.append((xmin, name))

        chars.sort(key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])
        lines.append(f'{image_path}\t{label}')

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(lines))

print(f'Done! {len(lines)} items written to {output_file}')


سلول 4 – آماده‌سازی فایل‌های GT برای validation

In [ ]:
valid_dir = '/content/Dataset/validation'
output_file = '/content/deep-text-recognition-benchmark/data/gt_validation.txt'
lines = []

for filename in os.listdir(valid_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(valid_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join('/content/deep-text-recognition-benchmark/data/validation', image_name)


        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            xmin = int(obj.find('bndbox').find('xmin').text)
            chars.append((xmin, name))

        chars.sort(key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])
        lines.append(f'{image_path}\t{label}')

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(lines))

print(f'Done! {len(lines)} items written to {output_file}')


سلول 5 – جابه‌جایی تصاویر به مسیر data/train و data/validation

In [ ]:
import shutil

# Move train images
src = '/content/Dataset/train'
dst = '/content/deep-text-recognition-benchmark/data/train'
os.makedirs(dst, exist_ok=True)

for file in os.listdir(src):
    if file.endswith('.jpg'):
        shutil.move(os.path.join(src, file), os.path.join(dst, file))

# Move validation images
src = '/content/Dataset/validation'
dst = '/content/deep-text-recognition-benchmark/data/validation'
os.makedirs(dst, exist_ok=True)

for file in os.listdir(src):
    if file.endswith('.jpg'):
        shutil.move(os.path.join(src, file), os.path.join(dst, file))

print("Train and validation images moved.")


🟢 سلول 6 – ساخت LMDB دیتاست

In [ ]:
%cd /content/deep-text-recognition-benchmark

!python3 create_lmdb_dataset.py --inputPath data/train --gtFile data/gt_train.txt --outputPath dataset/train
!python3 create_lmdb_dataset.py --inputPath data/validation --gtFile data/gt_validation.txt --outputPath dataset/validation

In [ ]:
!python3 train.py \
--train_data dataset/train --valid_data dataset/validation \
--select_data / --batch_ratio 1 --batch_max_length 8 --valInterval 100 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn


```
/content/
│
├── Dataset/
│   ├── train/
│   │   ├── *.jpg
│   │   └── *.xml
│   └── validation/
│       ├── *.jpg
│       └── *.xml
│
└── deep-text-recognition-benchmark/
    ├── data/
    │   ├── gt_train.txt
    │   ├── gt_validation.txt
    │   ├── train/         ← عکس‌های train
    │   └── validation/    ← عکس‌های validation
    └── dataset/
        ├── train/         ← LMDB ساخته شده
        └── validation/    ← LMDB ساخته شده

```


In [ ]:
def _accumulate(iterable):
    'Return running totals'
    total = 0
    for value in iterable:
        total += value
        yield total